In [1]:
#export
"""
This includes helper clis that make it quick to graph graphviz plots."""
__all__ = ["sketch", "edges"]
import re, k1lib, math, os, numpy as np, io, json, base64, unicodedata, inspect
from k1lib.cli.init import BaseCli; import k1lib.cli as cli, k1lib.cli.init as init
graphviz = k1lib.dep("graphviz")
from collections import deque, defaultdict
settings = k1lib.settings.cli
_svgAutoInc = k1lib.AutoIncrement(prefix="_k1_svg_")
_preAutoInc = k1lib.AutoIncrement(prefix="_k1_svg_pre_")
_clusterAuto = k1lib.AutoIncrement(prefix="cluster_")
_idxAuto = k1lib.AutoIncrement(prefix="_idx_")

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#export
class sketch(BaseCli):
    _g = None; _name2Idx = None; ctxIdx = None
    def __init__(self, **kwargs):
        """Similar to :class:`~utils.sketch`, which makes it easier to plot multiple graphs quickly,
this makes it easier to plot node graphs a lot quicker than I have before. This cli
configures the graph in general, but doesn't dive too much into the specifics. Example::

    ["ab", "bc", "ca"] | (kgv.sketch(engine="sfdp") | kgv.edges())

Most of the complexities are in :class:`edges`, so check that class out for more comprehensive examples

:param kwargs: keyword arguments passed into :class:`graphviz.Digraph`"""
        super().__init__(capture=True); self.kwargs = kwargs
    @staticmethod
    def _guard():
        if sketch._g is None: raise Exception("Context has not been setup yet, can't proceed to plot the edges/nodes. This could be because you're doing `data | kgv.edges()` directly. Instead, do `data | (kgv.sketch() | kgv.edges())`. The operation `kgv.sketch()` will initialize the graph")
    def __ror__(self, it):
        sketch._g = g = graphviz.Digraph(**self.kwargs); sketch._idx2Popup = idx2Popup = {}
        sketch._name2Idx = name2Idx = defaultdict(lambda: defaultdict(lambda: _idxAuto()))
        it | self.capturedSerial | cli.deref()
        sketch._g = None; sketch._idx2Popup = None; sketch._name2Idx = None; return Graph(g, name2Idx, idx2Popup)
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto(); ctxIdx = init._jsDAuto(); sketch.ctxIdx = ctxIdx
        header, _fIdx, _async = k1lib.kast.asyncGuard(self.capturedSerial._jsF(meta))
        res = f"""\
let {ctxIdx} = null;\n{header}
const {fIdx} = async ({dataIdx}) => {{
    {ctxIdx} = [];
    const out = {'await ' if _async else ''}{_fIdx}({dataIdx});

    const res = (await (await fetch("https://local.mlexps.com/routeServer/kapi_10-graphviz", {{
      method: "POST",
      body: JSON.stringify({{ "obj": {ctxIdx} }}),
      headers: {{ "Content-Type": "application/json" }}
    }})).json());
    if (!res.success) throw new Error(res.reason);
    return atob(res.data);
}}""", fIdx
        sketch.ctxIdx = None; return res

In [3]:
#export
class nodes(BaseCli):
    def __init__(self):
        pass
    def __ror__(self, _it) -> "5-column input":
        if sketch._g is None: return _it | (sketch() | self)
        sketch._guard(); it = []
        for row in _it:
            n = len(row)
            if n == 1: it.append(["", row[0], {}])
            elif n == 2: it.append([*row, {}])
            elif n == 3: it.append(row)
            else: raise Exception(f"kgv.nodes() can only accept tables from 1 to 3 columns. Detected {n} columns instead")
        g = sketch._g; name2Idx = sketch._name2Idx
        for s1,n1,kw in it:
            idx = name2Idx[s1][n1]
            if "popup" in kw: sketch._idx2Popup[idx] = kw["popup"]; kw = dict(kw); del kw["popup"]
            g.node(idx, **kw)
        return it
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto(); ctxIdx = sketch.ctxIdx;
        if ctxIdx is None: return (sketch() | self)._jsF(meta)
        return f"""\
const {fIdx} = ({dataIdx}) => {{
    const it = [];
    for (const row of {dataIdx}) {{
        const n = row.length;
        if (n === 1) it.push(["", row[0], {{}}])
        else if (n === 2) it.push([...row, {{}}])
        else if (n === 3) it.push(row)
        else throw new Error(`kgv.nodes() can only accept tables from 1 to 3 columns. Detected ${{n}} columns instead`)
    }}
    {ctxIdx}.push(["nodes", {{it, args: []}}]); return it;
}}
""", fIdx

In [4]:
#export
def drawSimple(g, names, data, name2Idx): # data here is List[[name1, name2, kw]], names is List[name]
    for name in set(names): g.node(name2Idx[name], name);
    for name1, name2, kw in data: g.edge(name2Idx[name1], name2Idx[name2], **kw)
class edges(BaseCli):
    def __init__(self):
        """Plots out edges of the graph.
Example 1::

    ["ab", "bc", "ca"] | kgv.edges()

Result:

.. image:: ../images/kgv_edges_1.png

If you need to customize the graph on initialization, then you can use :class:`sketch` to
capture related operations (like :class:`edges`), and inject your params in :class:`sketch`::

    ["ab", "bc", "ca"] | (kgv.sketch(engine="sfdp") | kgv.edges())

Example 2::

    [["a", "b", {"label": "3%"}], ["b", "c", {"color": "red"}], "ac", "cb"] | kgv.edges()

Result:

.. image:: ../images/kgv_edges_2.png

Example 3::

    [["group1", "a", "group1", "b", {"label": "3%"}],
     "ec",
     ["group1", "b", "", "c", {"color": "red"}],
     ["group1", "a", "", "c"],
     ["", "c", "group1", "b"],
     ["", "c", "group2", "d", {"color": "green"}]
    ] | kgv.edges()

Result:

.. image:: ../images/kgv_edges_3.png

So the idea is, each row describes a single edge on the graph. Each row
can be multiple different lengths, but only these configurations are allowed:

- [name1, name2]
- [name1, name2, kwargs]
- [group1, name1, group2, name2]
- [group1, name1, group2, name2, kwargs]

So if you don't need the complexity and just want to plot something out, you
can just use the one at the top, but if you do want fancy features, then you
can add those in the kwargs.

Check out a gallery of more examples at `kapi/10-graphviz <https://mlexps.com/kapi/10-graphviz/>`_.
"""
        pass
    def __ror__(self, _it) -> "5-column input":
        if sketch._g is None: return _it | (sketch() | self)
        sketch._guard(); it = []
        for row in _it:
            n = len(row)
            if n == 2: it.append(["", row[0], "", row[1], {}])
            elif n == 3: it.append(["", row[0], "", row[1], row[2]])
            elif n == 4: it.append([*row, {}])
            elif n == 5: it.append(row)
            else: raise Exception(f"kgv.edges() can only accept tables from 2 to 5 columns. Detected {n} columns instead")
        g = sketch._g; name2Idx = sketch._name2Idx
        # grouping by segments and drawing their internals first
        for segN, names in it | cli.batched(2).all() | cli.joinSt() | cli.groupBy(0, True) | cli.apply(cli.joinSt(), 1) | cli.deref():
            if segN:
                with g.subgraph(name=_clusterAuto()) as subG:
                    subG.attr(label=f"{segN}"); drawSimple(subG, names, it | cli.filt(cli.op() == segN, [0, 2]) | cli.cut(1, 3, 4) | cli.deref(), name2Idx[segN])
            else: drawSimple(g, names, it | cli.filt(cli.op() == segN, [0, 2]) | cli.cut(1, 3, 4) | cli.deref(), name2Idx[""])
        # then draw external edges
        for s1, n1, s2, n2, kw in it | cli.filt(lambda x: x[0] != x[2]): g.edge(name2Idx[s1][n1], name2Idx[s2][n2], **kw)
        return it
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto(); ctxIdx = sketch.ctxIdx;
        if ctxIdx is None: return (sketch() | self)._jsF(meta)
        return f"""\
const {fIdx} = ({dataIdx}) => {{
    // why not just pass dataIdx in directly? Well, in Python, the interface is that __ror__ should return a 5-column input, so here, gotta honor that, in case the user has some operation downstream of this
    const it = [];
    for (const row of {dataIdx}) {{
        const n = row.length;
        if (n === 2) it.push(["", row[0], "", row[1], {{}}])
        else if (n === 3) it.push(["", row[0], "", row[1], row[2]])
        else if (n === 4) it.push([...row, {{}}])
        else if (n === 5) it.push(row)
        else throw new Error(`kgv.edges() can only accept tables from 2 to 5 columns. Detected ${{n}} columns instead`)
    }}
    {ctxIdx}.push(["edges", {{it, args: []}}]); return it;
}}
""", fIdx

In [58]:
#export
class Graph:
    def __init__(self, g, name2Idx, idx2Popup):
        """Wrapper around a :class:`graphviz.Graph` or :class:`graphviz.Digraph`. Internal graph
object is available at ``self.g``. Not instantiated by end user, instead, returned by :class:`sketch`."""
        self.g = g; self.name2Idx = name2Idx; self.idx2Popup = idx2Popup
    def _repr_mimebundle_(self, *args, **kwargs): return self.g._repr_mimebundle_(*args, **kwargs)
    def _repr_html_(self): return self._toHtml()
    def _toHtml(self):
        idx2Popup = self.idx2Popup; name2Idx = self.name2Idx; s = self.g | cli.toHtml() | cli.op().replace("><", ">\n<")
        # these ids are: List[(auto generated id, unique id we're replacing it with)]
        # nodeIds = s.split("\n") | grep('<g id="(?P<g>node[0-9]+)"', extract="g") | apply(lambda x: [x, _svgAutoInc()]) | deref()
        nodeIds = s.split("\n") | cli.grep('class="node"', after=1) | cli.batched(2) | cli.apply(cli.op().split("title>")[1].strip("</"), 1) | cli.grep('<g id="(?P<g>node[0-9]+)', col=0, extract="g") | cli.deref()
        edgeIds = s.split("\n") | cli.grep('<g id="(?P<g>edge[0-9]+)"', extract="g") | cli.apply(lambda x: [x, _svgAutoInc()]) | cli.deref()
        graphIds = s.split("\n") | cli.grep('<g id="(?P<g>graph[0-9]+)"', extract="g") | cli.apply(lambda x: [x, _svgAutoInc()]) | cli.deref()
        for x, y in [nodeIds, edgeIds, graphIds] | cli.joinSt(): s = s.replace(f'id="{x}"', f'id="{y}"')
        a = nodeIds | cli.cut(1) | cli.apply(lambda idx: [idx, idx2Popup.get(idx, None)]) | cli.deref(); pre = _preAutoInc()
        inside = f"rect.x <= {pre}_mouseX && {pre}_mouseX < rect.x+rect.width && rect.y <= {pre}_mouseY && {pre}_mouseY < rect.y+rect.height"
        return f"""
<div id="{pre}_wrapper" style="position: relative">{s}<div id="{pre}_popup" style="position: absolute; display: none; background: white; padding: 8px 12px; border-radius: 6px; box-shadow: 0 3px 5px rgb(0 0 0 / 0.3); z-index: 1000000"></div></div>
<script>
    const {pre}_nodeId_node_popup = ({json.dumps(a)}).map(([x,y]) => [x, document.querySelector(`#${{x}}`), y]);
    const {pre}_nodes = {pre}_nodeId_node_popup.map(([x,n,y]) => n); let {pre}_activeNode = null;
    const {pre}_popup = document.querySelector("#{pre}_popup"); const {pre}_wrapper = document.querySelector("#{pre}_wrapper");
    const {pre}_nodeId2popup = {{}}; for (const [x,n,y] of {pre}_nodeId_node_popup) {{ {pre}_nodeId2popup[x] = y; }};
    let {pre}_mouseX = 0; let {pre}_mouseY = 0; {pre}_wrapper.onmousemove = (e) => {{ {pre}_mouseX = e.clientX; {pre}_mouseY = e.clientY; }};

    setInterval(() => {{
        if ({pre}_activeNode) {{
            const rect = {pre}_activeNode.getBoundingClientRect();
            if (!({inside})) {{ {pre}_activeNode = null; {pre}_popup.innerHTML = ""; {pre}_popup.style.display = "none"; }}
        }}

        if (!{pre}_activeNode) {{ // can't just do `if (activeNode) ... else ...` btw. Separated out for a reason
            const wRect = {pre}_wrapper.getBoundingClientRect();
            for (const node of {pre}_nodes) {{
                const rect = node.getBoundingClientRect();
                if ({inside}) {{
                    const popup = {pre}_nodeId2popup[node.id];
                    {pre}_activeNode = node;
                    if (popup) {{
                        {pre}_popup.style.left = rect.x + rect.width/2 + 10 - wRect.x + "px";
                        {pre}_popup.style.top = 0; {pre}_popup.innerHTML = popup; {pre}_popup.style.display = "block";
                        const pRect = {pre}_popup.getBoundingClientRect();
                        const t1 = rect.y + rect.height/2 + 10 - wRect.y; // "t" for "top"
                        const t2 = wRect.height - pRect.height;
                        {pre}_popup.style.top = ((t2 < 0) ? 0 : Math.min(t1, t2)) + "px";
                    }}
                    break;
                }}
            }}
        }}
    }}, 30);
</script>"""
        return s

In [23]:
import IPython

In [71]:
["ab", "bc", "ca"] | edges() | cli.toImg() | cli.file("../../docs/images/kgv_edges_1.png")
[["a", "b", {"label": "3%"}], ["b", "c", {"color": "red"}], "ac", "cb"] | edges() | cli.toImg() | cli.file("../../docs/images/kgv_edges_2.png")
[["group1", "a", "group1", "b", {"label": "3%"}], "ec", ["group1", "b", "", "c", {"color": "red"}], ["group1", "a", "", "c"], ["", "c", "group1", "b"], ["", "c", "group2", "d", {"color": "green"}]] | edges() | cli.toImg() | cli.file("../../docs/images/kgv_edges_3.png")

'../../docs/images/kgv_edges_3.png'

In [62]:
!../../export.py cli/kgv --upload=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-29 07:54:19,173	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-29 07:54:19,183	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13302   0   60%   
8859    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/

In [60]:
!../../export.py cli/kgv

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-28 17:51:23,301	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-28 17:51:23,319	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13281   0   60%   
8826    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/

In [ ]:
!../../export.py cli/kgv --boostrap=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-28 01:20:32,536	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-28 01:20:32,546	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13278   0   60%   
8824    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/